In [ ]:

from datetime import timedelta

import numpy as np
import pandas as pd
import plotly.graph_objects as go

from Config import config
from Model.Data.atr import read_multi_timeframe_ohlcva
from helper.data_preparation import single_timeframe
from helper.helper import date_range_to_string


# reading sample data
volume_rma = volume / ta.ram(volume_rma, 14)

In [ ]:
config.processing_date_range = date_range_to_string(start=pd.to_datetime('03-01-24'),
                                                    end=pd.to_datetime('09-01-24'))
mt_ohlcva = read_multi_timeframe_ohlcva(config.processing_date_range)
mt_ohlcva

In [ ]:
mt_ohlcva

# Visualizing rolling mean
open = open - rolling(mean, 256)

In [ ]:
fig = go.Figure()
count, edges = np.histogram(np.log(mt_ohlcva['open'] / mt_ohlcva['atr']), bins=100, )
n_probability = count / len(mt_ohlcva)
fig = go.Figure()
fig.add_trace(go.Scatter(x=edges, y=n_probability, mode='lines', name='p', line=dict(color='blue')))
fig.update_layout(title='Volume RMA Over Time', xaxis_title='Time', yaxis_title='Volume RMA', template='plotly_white')
fig.show()

# Visualizing rolling std
open = open / rolling(std, 256)

# visual compare_transformation

In [ ]:
from plotly.subplots import make_subplots
from random import randint, seed

np.random.seed(42)
seed(42)

price_columns = {
    'open': 'gray',
    'high': 'green',
    'low': 'red',
    'close': 'blue',
}


def trace_chart(i_fig, df, name_prefix, fig_row, fig_col):
    for col in price_columns.keys():
        x = df.index.get_level_values('date')
        i_fig.add_trace(
            go.Scatter(x=x, y=df[col], mode='lines', name=f'{name_prefix} {col}', line=dict(color=price_columns[col])),
            row=fig_row, col=fig_col)
    i_fig.add_trace(
        go.Scatter(x=x, y=df['volume'], mode='lines', name=f'{name_prefix} volume', line=dict(color='orange')),
        row=fig_row, col=fig_col)
    return i_fig


def visualize_transformation(orig_ohlcv, trans_original_ohlcv):
    duration = 256  # candles
    start = trans_original_ohlcv.index.get_level_values('date').min()
    end = trans_original_ohlcv.index.get_level_values('date').max() - timedelta(days=1)
    for timeframe in trans_original_ohlcv.index.get_level_values(level='timeframe').unique():
        n_ohlcva = single_timeframe(trans_original_ohlcv, timeframe)
        ohlcva = single_timeframe(orig_ohlcv, timeframe)
        for random_start_day in np.random.choice(pd.date_range(start, end, freq=timedelta(days=1)), size=1):
            random_start = random_start_day + timedelta(seconds=randint(0, 60 * 60 * 24))
            t_n_ohlcva = n_ohlcva[
                (n_ohlcva.index.get_level_values('date') >= random_start) &
                (n_ohlcva.index.get_level_values('date') < random_start + (duration * pd.to_timedelta(timeframe)))]
            t_ohlcva = ohlcva[
                (ohlcva.index.get_level_values('date') >= random_start) &
                (ohlcva.index.get_level_values('date') < random_start + duration * pd.to_timedelta(timeframe))]
            fig = make_subplots(rows=2, cols=1, shared_xaxes=True,
                                vertical_spacing=0.2, subplot_titles=("transformed", "original"))
            fig = trace_chart(fig, t_n_ohlcva, "transformed", fig_row=1, fig_col=1)
            fig = trace_chart(fig, t_ohlcva, "original", fig_row=2, fig_col=1)

            fig.update_layout(
                title=f'Timeframe:{timeframe}, Start:{random_start}, Duration:{duration * pd.to_timedelta(timeframe)} ',
                height=800, xaxis_rangeslider=dict(visible=False),
                xaxis2_rangeslider=dict(
                    visible=True,
                    yaxis=dict(rangemode='auto',
                               range=[0, 0.3],  # Adjust the range height here to reduce the height
                               ),
                    thickness=0.05,  # Control the thickness of the range selector (default is 0.15)
                ), yaxis=dict(fixedrange=False), yaxis2=dict(fixedrange=False),  # vertical zooming
            ).show()
            # html_file_path = f"{name}.html"
            # fig.write_html(html_file_path)
            # if os.path.exists(html_file_path):
            #     display(IFrame(src=html_file_path, width='100%', height=800))

# compare_transformation(mt_ohlcva, mt_ohlcva)

# np.log transformation

# volume_rma np.log transform

In [ ]:
t = mt_ohlcva.copy()
t['n_volume_rma'] = np.log(t['volume_rma'] + 0.0000000000000000000000000001)
fig = go.Figure()
n_probability = count / len(t)
fig = go.Figure()
fig.add_trace(go.Scatter(x=edges, y=n_probability, mode='lines', name='p', line=dict(color='blue')))
fig.update_layout(title='log Volume RMA PDF', xaxis_title='Time', yaxis_title='Volume RMA',
                  template='plotly_white').show()
t['volume_rma'].describe()

In [ ]:
n_mt_ohlcva = mt_ohlcva[['open', 'close', 'high', 'low', 'volume']].copy()
n_mt_ohlcva.describe()

# Shift Transform

In [ ]:
before = n_mt_ohlcva.copy()

In [ ]:
shifted_mt_ohlcva = n_mt_ohlcva.shift(1)
for timeframe in n_mt_ohlcva.index.get_level_values('timeframe').unique():
    shifted_mt_ohlcva = n_mt_ohlcva.loc[pd.IndexSlice[timeframe, :]].copy().shift(1)
    n_mt_ohlcva.loc[pd.IndexSlice[timeframe, :], ['open', 'close', 'high', 'low']] = (
                n_mt_ohlcva.loc[pd.IndexSlice[timeframe, :], ['open', 'close', 'high', 'low']]
                - shifted_mt_ohlcva[['open', 'close', 'high', 'low']])
visualize_transformation(before, n_mt_ohlcva, )
n_mt_ohlcva.describe()

# ATR transform

In [ ]:
before = n_mt_ohlcva.copy()

In [ ]:
n_mt_ohlcva[['open', 'close', 'high', 'low']] = n_mt_ohlcva[['open', 'close', 'high', 'low']].div(mt_ohlcva['atr'],
                                                                                                  axis=0)
n_mt_ohlcva['volume'] = mt_ohlcva['volume_rma']
n_mt_ohlcva['atr'] = mt_ohlcva['atr']
visualize_transformation(before, n_mt_ohlcva)
n_mt_ohlcva.describe()


# np.log transformation

In [ ]:
before = n_mt_ohlcva.copy()

In [ ]:
n_mt_ohlcva[['open', 'close', 'high', 'low']] = np.log(
    n_mt_ohlcva[['open', 'close', 'high', 'low']] + 0.00000000000000000000000001)
visualize_transformation(before, n_mt_ohlcva)
n_mt_ohlcva.describe()


# standard transformation

In [ ]:
from sklearn.preprocessing import StandardScaler

n_mt_ohlcva = mt_ohlcva.copy()
transformers = {'volume': StandardScaler()}
for col in ['open', 'close', 'high', 'low']:
    n_mt_ohlcva[col] = n_mt_ohlcva[col] / n_mt_ohlcva['atr']
    print(f"original: n_mt_ohlcva[{col}]:" + str(n_mt_ohlcva[col].describe()[['mean', 'std']].to_dict()))
    n_mt_ohlcva[col] = np.log(n_mt_ohlcva[col])
    print(f"np.log: n_mt_ohlcva[{col}]:" + str(n_mt_ohlcva[col].describe()[['mean', 'std']].to_dict()))
    transformers[col] = StandardScaler()
    n_mt_ohlcva[col] = transformers[col].fit_transform(n_mt_ohlcva[[col]])
    print(f"fit_transform: n_mt_ohlcva[{col}]:" + str(n_mt_ohlcva[col].describe()[['mean', 'std']].to_dict()))


for random_start in

In [ ]:


visualize_transformation(mt_ohlcva, n_mt_ohlcva, "mt_ohlcva vs n_mt_ohlcva")